# Environment set up

In [2]:
!pip install numpy matplotlib scipy numba scikit-learn mne PyWavelets pandas autoreject mne-features torcheeg

In [3]:
import os
import mne
import numpy as np
import pandas as pd
from autoreject import AutoReject
import timeit
import torch
import matplotlib.pyplot as plt
from IPython.display import clear_output

# Get data

In [4]:
# sample data
# import gdown
# !gdown 1RCHit--cMXX-qde5i0kHmJuSnEaQMGGr

In [5]:
# real data TDBRAIN
# !gdown 1LxJtBcaXYsB9Q306ysaZzBF7vObMaktt
# !unzip -P '!Bra1n$Rgr3at:)' /content/TDBRAIN-dataset.zip

In [6]:
# metadata
# !gdown --folder 1tDXSxIBMlnUZU6wADPZSAt9pHQ2znDvb

# Data Exploration

## Sample data

## Real data

In [10]:
metadata = pd.read_csv('/home/ubuntu/nss/nss_data/DORTMUND/BrainAge_v1.0_ohneDKA.csv', sep=';')
pd.set_option('display.max_columns', None)
metadata['gender'] = [0 if s == "f" else 1 for s in metadata.sex]
metadata

,VP_ID,sex,age_jittered,BMI,MBI_total,OLBI_total,PSQ_worries,PSQ_tension,PSQ_joy,PSQ_demands,PSQ_total,attention,verbal_fluency,verbal_memory,working_memory,g_intelligence,LPAQ_sport,LPQA_total,WAI1_current,WAI2_demands,WAI3_diseases,WAI4_impairment,WAI5_sick_leave,WAI6_prognosis,WAI7_resources,WAI_total,gender
0,VP001,m,21,25.51,,,0,20,86.67,40,18.33,-.32581493,-.8419919,-.67443979,-.52770917,-.60135415,17.5,22.2,,,7,,,,,,1
1,VP002,w,21,22.89,18,11,26.67,26.67,53.33,33.33,33.33,.32204864,-.44636074,.56280821,-.01906312,-.15326894,9.3,9.8,10,10,5,6,4,7,3,45,1
2,VP003,w,21,23.67,17,11,0,26.67,73.33,26.67,20,.45412219,.42936051,-.27568764,.19469877,.24884916,9,14,10,,5,6,4,7,4,36,1
3,VP004,m,21,43.38,24,13,80,53.33,53.33,33.33,53.33,-.31436955,-.55297194,-.71458645,-.44480072,-.44593893,0,7.7,9,10,5,6,5,7,3,45,1
4,VP005,w,21,19.76,31,13,13.33,46.67,66.67,46.67,35,-.07429391,.30132801,-.17627548,.26545035,.18316646,0,2.4,9,10,5,6,4,7,3,44,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
613,VP614,m,68,24.47,,,13.33,58.33,46.67,26.67,36.84,-.67209501,-.62282624,-.48248236,-.50148306,-.48922921,6,7,,,1,,,,,,1
614,VP615,m,68,30.53,,,13.33,6.67,93.33,33.33,15,-.96567945,-.64974459,-.45724157,.1428507,-.37862606,11,22.5,,,1,,,,,,1
615,VP616,m,69,33.48,,,20,6.67,86.67,20,15,.49953315,.37805994,.96402826,.12401964,.32515553,0,7.5,,,1,,,,,,1
616,VP617,w,69,25.09,,,13.33,6.67,86.67,26.67,15,.52979073,.54054252,.42848459,.25004819,.3834503,4.5,16,,,1,,,,,,1


In [11]:
metadata = metadata[["VP_ID", "gender"]].dropna(how="any")
metadata

,VP_ID,gender
0,VP001,1
1,VP002,1
2,VP003,1
3,VP004,1
4,VP005,1
...,...,...
613,VP614,1
614,VP615,1
615,VP616,1
616,VP617,1


In [24]:
#select only session 1 so that repeated session for one person would not be in analysis?
all_ses1_files = []
for root, dirs, files in os.walk("/home/ubuntu/nss/nss_data/DORTMUND/raw"):
    for file in files:
        if file.endswith(".set"): #filtered data and closed eyes
             all_ses1_files.append(os.path.join(root, file))

for path in all_ses1_files:
    # print(path[39:44])
    index = metadata.loc[metadata["VP_ID"]==path[39:44]].index
    if len(index) > 0:
        metadata.loc[metadata.index[index[0]], 'path'] = path


['/home/ubuntu/nss/nss_data/DORTMUND/raw/VP076_RestState.set',
 '/home/ubuntu/nss/nss_data/DORTMUND/raw/VP092_RestState.set',
 '/home/ubuntu/nss/nss_data/DORTMUND/raw/VP259_RestState.set',
 '/home/ubuntu/nss/nss_data/DORTMUND/raw/VP217_RestState.set',
 '/home/ubuntu/nss/nss_data/DORTMUND/raw/VP141_RestState.set',
 '/home/ubuntu/nss/nss_data/DORTMUND/raw/VP101_RestState.set',
 '/home/ubuntu/nss/nss_data/DORTMUND/raw/VP044_RestState.set',
 '/home/ubuntu/nss/nss_data/DORTMUND/raw/VP239_RestState.set',
 '/home/ubuntu/nss/nss_data/DORTMUND/raw/VP165_RestState.set',
 '/home/ubuntu/nss/nss_data/DORTMUND/raw/VP208_RestState.set',
 '/home/ubuntu/nss/nss_data/DORTMUND/raw/VP074_RestState.set',
 '/home/ubuntu/nss/nss_data/DORTMUND/raw/VP083_RestState.set',
 '/home/ubuntu/nss/nss_data/DORTMUND/raw/VP115_RestState.set',
 '/home/ubuntu/nss/nss_data/DORTMUND/raw/VP031_RestState.set',
 '/home/ubuntu/nss/nss_data/DORTMUND/raw/VP238_RestState.set',
 '/home/ubuntu/nss/nss_data/DORTMUND/raw/VP145_RestStat

In [26]:
#select only session 1 so that repeated session for one person would not be in analysis?
all_ses1_files = []
for root, dirs, files in os.walk("/home/ubuntu/nss/nss_data/DORTMUND/raw"):
    for file in files:
        if file.endswith(".set"): #filtered data and closed eyes
             all_ses1_files.append(os.path.join(root, file))

for path in all_ses1_files:
    # print(path[39:44])
    index = metadata.loc[metadata["VP_ID"]==path[39:44]].index
    if len(index) > 0:
        metadata.loc[metadata.index[index[0]], 'path'] = path

#gender_list = []
#for participant in participants_list:
    #a = (metadata.loc[(metadata["subID"]) == participant, "gender"]).values
   # if len(a) > 0:
       # gender_list.append(int(a[0]))
   # else:
       # print(participants_list.index(participant))
       # epochs.remove(epochs[participants_list.index(participant)])

metadata_list_d = []
for i in range(metadata.shape[0]):
    metadata_dictionary = {}
    metadata_dictionary['subject'] = metadata["VP_ID"][i]
    metadata_dictionary['gender'] = metadata["gender"][i]
    metadata_list_d.append(metadata_dictionary)

metadata_list_d[:10]    
# metadata_list_d
# separate experiments with closed and open eyes
# ses1_closed_eyes = []
# ses1_open_eyes = []
# for i in all_ses1_files:
#   if "EC" in i:
#     ses1_closed_eyes.append(i)
#   else:
#     ses1_open_eyes.append(i)

[{'subject': 'VP001', 'gender': 1},
 {'subject': 'VP002', 'gender': 1},
 {'subject': 'VP003', 'gender': 1},
 {'subject': 'VP004', 'gender': 1},
 {'subject': 'VP005', 'gender': 1},
 {'subject': 'VP006', 'gender': 1},
 {'subject': 'VP007', 'gender': 1},
 {'subject': 'VP008', 'gender': 1},
 {'subject': 'VP009', 'gender': 1},
 {'subject': 'VP010', 'gender': 1}]

In [10]:
# helper = pd.read_table('/home/ubuntu/nss/nss_data/TD-BRAIN/TD-BRAIN-DATASET/Processed-BC-notch_50_full/88079017/88079017-1/88079017-1_EO_eeg.tsv')
# ch_names = list(helper.name[:26])

# info = mne.create_info(ch_names = ch_names, ch_types='eeg', sfreq = 500)

In [29]:
metadata.path

0      /home/ubuntu/nss/nss_data/DORTMUND/raw/VP001_R...
1      /home/ubuntu/nss/nss_data/DORTMUND/raw/VP002_R...
2                                                    NaN
3      /home/ubuntu/nss/nss_data/DORTMUND/raw/VP004_R...
4      /home/ubuntu/nss/nss_data/DORTMUND/raw/VP005_R...
                             ...                        
613                                                  NaN
614                                                  NaN
615                                                  NaN
616                                                  NaN
617                                                  NaN
Name: path, Length: 618, dtype: object

In [30]:
dortmund = []
# channels_to_drop = ["VPVA",	"VNVB",	"HPHL",	"HNHR",	"Erbs",	"OrbOcc",	"Mass"]

for i, file in enumerate(metadata.dropna().path):
    # df = np.load(all_ses1_files[i])
    # df = df[:(len(ch_names)), :] #
    print(file)
    raw = mne.io.read_raw_eeglab(file) #, info, first_samp=0, copy='auto', verbose=None)
    raw.resample(sfreq=250)
    dortmund.append(raw)

/home/ubuntu/nss/nss_data/DORTMUND/raw/VP001_RestState.set


/tmp/ipykernel_466550/3262067425.py:8: RuntimeWarning: Data will be preloaded. preload=False or a string preload is not supported when the data is stored in the .set file
  raw = mne.io.read_raw_eeglab(file) #, info, first_samp=0, copy='auto', verbose=None)
/tmp/ipykernel_466550/3262067425.py:8: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(file) #, info, first_samp=0, copy='auto', verbose=None)


/home/ubuntu/nss/nss_data/DORTMUND/raw/VP002_RestState.set


/tmp/ipykernel_466550/3262067425.py:8: RuntimeWarning: Data will be preloaded. preload=False or a string preload is not supported when the data is stored in the .set file
  raw = mne.io.read_raw_eeglab(file) #, info, first_samp=0, copy='auto', verbose=None)
/tmp/ipykernel_466550/3262067425.py:8: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(file) #, info, first_samp=0, copy='auto', verbose=None)


/home/ubuntu/nss/nss_data/DORTMUND/raw/VP004_RestState.set


...

/tmp/ipykernel_466550/3262067425.py:8: RuntimeWarning: Data will be preloaded. preload=False or a string preload is not supported when the data is stored in the .set file
  raw = mne.io.read_raw_eeglab(file) #, info, first_samp=0, copy='auto', verbose=None)
/tmp/ipykernel_466550/3262067425.py:8: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(file) #, info, first_samp=0, copy='auto', verbose=None)


/home/ubuntu/nss/nss_data/DORTMUND/raw/VP269_RestState.set


/tmp/ipykernel_466550/3262067425.py:8: RuntimeWarning: Data will be preloaded. preload=False or a string preload is not supported when the data is stored in the .set file
  raw = mne.io.read_raw_eeglab(file) #, info, first_samp=0, copy='auto', verbose=None)
/tmp/ipykernel_466550/3262067425.py:8: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(file) #, info, first_samp=0, copy='auto', verbose=None)


/home/ubuntu/nss/nss_data/DORTMUND/raw/VP270_RestState.set


/tmp/ipykernel_466550/3262067425.py:8: RuntimeWarning: Data will be preloaded. preload=False or a string preload is not supported when the data is stored in the .set file
  raw = mne.io.read_raw_eeglab(file) #, info, first_samp=0, copy='auto', verbose=None)
/tmp/ipykernel_466550/3262067425.py:8: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(file) #, info, first_samp=0, copy='auto', verbose=None)


In [31]:
dortmund[0]

<RawEEGLAB | VP001_RestState.set, 64 x 180000 (720.0 s), ~88.0 MB, data loaded>

In [32]:
dortmund[0].to_data_frame()

,time,Fp1,Fp2,F7,F3,Fz,F4,F8,FC5,FC1,FC2,FC6,T7,C3,Cz,C4,T8,TP9,CP5,CP1,CP2,CP6,TP10,P7,P3,Pz,P4,P8,PO9,O1,Oz,O2,PO10,AF7,AF3,AF4,AF8,F5,F1,F2,F6,FT9,FT7,FC3,FC4,FT8,FT10,C5,C1,C2,C6,TP7,CP3,CPz,CP4,TP8,P5,P1,P2,P6,PO7,PO3,POz,PO4,PO8
0,0.000,534.004589,947.010465,13.515565,124.510283,163.498868,614.008818,-129.496852,57.505586,-122.504990,-119.498180,72.498096,-246.483590,18.989830,-94.000113,-445.990676,-397.999139,-78.506172,-710.495167,-182.508399,-262.994469,-423.988830,258.504581,-361.995831,-317.001195,-455.498857,-418.985459,-652.483563,-161.501320,80.994471,298.999279,-610.488285,-620.496343,455.011054,-288.498106,448.006387,75.483931,201.002113,1056.993523,-517.499065,739.006081,193.020260,-211.485124,109.986456,-240.490996,127.502724,155.507584,-692.491808,74.494499,12.004974,-978.981845,276.509433,-5.508301,615.492472,-119.988119,35.507515,472.494094,481.993321,-608.489460,-448.486129,-416.998951,636.491019,467.997753,-531.495299,249.501864
1,0.004,531.553643,936.515038,17.297118,124.424263,154.569673,595.200548,-151.519946,57.919030,-124.028612,-124.458294,67.055598,-243.562528,23.397054,-93.659352,-432.819324,-412.722653,-86.404656,-708.983310,-183.026786,-258.981477,-416.427818,253.709244,-355.548745,-311.382679,-455.268110,-415.600410,-651.084232,-154.303603,86.827094,305.546533,-605.010408,-616.352341,458.430092,-293.277209,432.481262,62.703007,203.185538,1054.714080,-524.748341,723.275644,193.517725,-208.102047,112.305274,-240.983179,96.778393,152.918210,-690.707921,75.368788,16.278164,-958.957285,282.607936,3.000004,618.890033,-112.447146,35.259721,479.738601,484.572608,-605.474932,-443.565591,-409.668689,643.865665,475.101801,-527.497547,256.091937
2,0.008,532.867373,939.298474,16.951114,120.957183,152.642338,605.577301,-136.121520,40.572106,-127.707228,-128.906884,64.523231,-242.029388,20.095489,-98.180579,-459.725015,-388.603018,-79.616143,-713.097291,-184.026924,-264.563833,-428.893265,263.458007,-359.354747,-312.968285,-458.852528,-419.724527,-649.454918,-166.385634,72.691165,292.829943,-609.446824,-611.974652,456.140832,-300.009600,437.987421,70.819770,198.377391,1050.814400,-527.312801,733.462375,196.962210,-205.456521,108.534552,-255.284991,129.418189,163.108089,-695.229050,72.300418,8.355941,-988.798353,279.114074,2.452685,615.592481,-122.817940,40.679041,480.677069,482.994990,-610.633911,-446.709114,-413.818357,639.146259,469.358941,-531.752354,255.423998
3,0.012,533.573847,938.181892,11.960288,120.278982,155.148676,602.458991,-137.348985,44.937998,-123.923988,-126.975696,64.869337,-248.592524,24.586199,-97.282718,-452.624774,-397.380844,-81.145650,-710.253018,-186.780731,-268.165642,-429.385487,263.558346,-367.078793,-319.044227,-462.829960,-427.395245,-653.232073,-170.899711,67.237618,286.475263,-619.159861,-619.114452,454.508891,-295.155027,439.147332,69.818260,198.010230,1054.424197,-525.686384,730.094174,192.756912,-213.138392,115.051002,-251.618018,123.033374,161.444491,-690.446736,74.034577,9.620864,-985.410883,289.784285,-2.702041,613.749452,-124.845901,40.936772,472.069374,478.558309,-613.505896,-454.026782,-419.896768,634.600093,465.589194,-535.898992,247.945107
4,0.016,543.124525,945.413062,24.154958,130.573349,162.304307,609.078752,-135.771848,69.482136,-118.909983,-122.111435,66.657627,-234.194356,24.807969,-93.524868,-447.515125,-393.861039,-84.467357,-705.256533,-182.349781,-263.108948,-423.893561,265.576755,-364.077253,-315.641590,-458.227716,-425.341281,-656.196598,-168.425282,68.116134,287.565580,-624.859578,-637.907117,463.943233,-287.823528,446.014902,71.534327,208.027732,1059.878268,-518.959025,734.676910,201.299319,-198.835462,116.240450,-249.060347,121.329212,163.665052,-688.746538,76.822237,11.801028,-981.440939,291.098192,0.703032,614.032047,-123.320345,33.847760,472.259417,481.366395,-612.145516,-453.099505,-420.375456,635.418861,464.470657,-540.217371,239.551567
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

In [33]:
dortmund[0].compute_psd(fmax=100).plot(picks="data", exclude="bads")
# tdbrain[0].plot(duration=2, n_channels=26)

Effective window size : 8.192 (s)


AttributeError: 'NoneType' object has no attribute '_get_renderer'

<MNELineFigure size 1000x350 with 2 Axes>

AttributeError: 'NoneType' object has no attribute '_get_renderer'

<MNELineFigure size 1000x350 with 2 Axes>

In [34]:
#creating epochs
epochs = []
for i in dortmund:
    epochs.append(mne.make_fixed_length_epochs(i, duration=4.0, overlap=0.5, preload=True, verbose=None))

#filtering epochs
#epochs_clean = []
#ar = AutoReject()
#for i in epochs:
  #epochs_clean.append(ar.fit_transform(i))

Not setting metadata
205 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 205 events and 1000 original time points ...
0 bad epochs dropped
Not setting metadata
205 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 205 events and 1000 original time points ...
0 bad epochs dropped
Not setting metadata
205 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 205 events and 1000 original time points ...
0 bad epochs dropped
Not setting metadata
205 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 205 events and 1000 original time points ...
0 bad epochs dropped
Not setting metadata
205 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 205 events and 1000 original ti

In [35]:
epochs[0]

Number of events,205
Events,1: 205
Time range,0.000 – 3.996 s
Baseline,off


# Features

# Creating basic model

In [36]:
import logging
import os
import random
import time

import mne
import numpy as np
import torch
from torch.utils.data.dataloader import DataLoader

from torcheeg import transforms
from torcheeg.datasets import MNEDataset
from torcheeg.model_selection import KFold, KFoldPerSubject, LeaveOneSubjectOut, KFoldCrossSubject
from torcheeg.models import EEGNet, TSCeption, GRU, ViT
from torcheeg.trainers import ClassificationTrainer, ClassifierTrainer

In [37]:
os.getcwd()

'/home/ubuntu/nss/depression/models/GNNs'

In [38]:
os.makedirs('./tmp_out_dortmund/examples_mne_dataset/log', exist_ok=True)
logger = logging.getLogger('Examples of MNEDataset')
logger.setLevel(logging.DEBUG)
console_handler = logging.StreamHandler()
timeticks = time.strftime("%Y-%m-%d-%H-%M-%S", time.localtime())
file_handler = logging.FileHandler(
    os.path.join('./tmp_out_dortmund/examples_mne_dataset/log', f'{timeticks}.log'))
logger.addHandler(console_handler)
logger.addHandler(file_handler)

In [39]:
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


seed_everything(42)

In [40]:
class MyClassificationTrainer(ClassificationTrainer): #MyClassificationTrainer(ClassificationTrainer):
    def log(self, *args, **kwargs):
        if self.is_main:
            logger.info(*args, **kwargs)

In [41]:
len(metadata_list_d), len(epochs)

(618, 249)

In [42]:
!rm -rf ./tmp_out_dortmund/examples_mne_dataset/dortmund
dataset = MNEDataset(epochs_list=epochs,
                     metadata_list=metadata_list_d,
                     io_path='./tmp_out_dortmund/examples_mne_dataset/dortmund',
                     chunk_size=250,
                     overlap=125,
                     io_size=20000000000,
                     offline_transform=transforms.Compose(
                         [transforms.MeanStdNormalize(),
                          transforms.To2d()]),
                     online_transform=transforms.ToTensor(),
                     label_transform=transforms.Compose([
                         transforms.Select('gender')
                         ]),
                     num_worker=2)

AssertionError: The number of mne.Epochs 249 does not match the number of metadata 618.

In [ ]:
print(dataset[0])
    # EEG signal (torch.Tensor[60, 4]),
    # coresponding baseline signal (torch.Tensor[60, 4]),
    # label (int)

In [58]:
dataset

MNEDataset(
    io_path='./tmp_out/examples_mne_dataset/tdbrain',
    io_size=10000000000,
    io_mode='lmdb',
    in_memory=False,
    chunk_size=250,
    overlap=125,
    num_channel=-1,
    online_transform=ToTensor(apply_to_baseline=False),
    offline_transform=Compose(
    MeanStdNormalize(apply_to_baseline=False, mean=None, std=None, axis=None),
    To2d(apply_to_baseline=False)
),
    label_transform=Compose(
    Select(key='gender')
),
    before_trial=None,
    after_trial=None,
    num_worker=2,
    verbose=True
)
length=142800

In [59]:
# k_fold = KFold(n_splits=5, split_path='./tmp_out/examples_mne_dataset/split')

In [60]:
k_fold_subj = KFoldCrossSubject(n_splits=5, shuffle=True, split_path='./tmp_out/examples_mne_dataset/split')

In [61]:
# !ls ./tmp_out/examples_mne_dataset/split

In [62]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [64]:
for i, (train_dataset, val_dataset) in enumerate(k_fold_subj.split(dataset)):
    # Initialize the model

    model = TSCeption(num_electrodes=26,
                      num_classes=2,
                      num_T=15,
                      num_S=15,
                      in_channels=1,
                      hid_channels=32,
                      sampling_rate=250,
                      dropout=0.5)

    # Initialize the trainer and use the 0-th GPU for training, or set device_ids=[] to use CPU
    trainer = MyClassificationTrainer(model=model,
                                      lr=1e-4,
                                      weight_decay=1e-4,
                                      device_ids=[0])

    # Initialize several batches of training samples and test samples
    train_loader = DataLoader(train_dataset,
                              batch_size=25,
                              shuffle=True,
                              num_workers=6)
    val_loader = DataLoader(val_dataset,
                            batch_size=25,
                            shuffle=False,
                            num_workers=6)

    # Do 50 rounds of training
    trainer.fit(train_loader, val_loader, num_epochs=100)
    trainer.test(val_loader)


Epoch 1
-------------------------------
Epoch 1
-------------------------------
loss: 0.656305, accuracy: 68.0% [    0/  191]
loss: 0.656305, accuracy: 68.0% [    0/  191]
loss: 0.663142, accuracy: 56.0% [   39/  191]
loss: 0.663142, accuracy: 56.0% [   39/  191]
loss: 0.650657, accuracy: 60.0% [   78/  191]
loss: 0.650657, accuracy: 60.0% [   78/  191]
loss: 0.613078, accuracy: 56.0% [  117/  191]
loss: 0.613078, accuracy: 56.0% [  117/  191]
loss: 0.552573, accuracy: 80.0% [  156/  191]
loss: 0.552573, accuracy: 80.0% [  156/  191]

loss: 0.715956, accuracy: 60.0%

loss: 0.715956, accuracy: 60.0%
Epoch 2
-------------------------------
Epoch 2
-------------------------------
loss: 0.583749, accuracy: 60.0% [    0/  191]
loss: 0.583749, accuracy: 60.0% [    0/  191]
loss: 0.580742, accuracy: 68.0% [   39/  191]
loss: 0.580742, accuracy: 68.0% [   39/  191]
loss: 0.535611, accuracy: 72.0% [   78/  191]
loss: 0.535611, accuracy: 72.0% [   78/  191]
loss: 0.540561, accuracy: 64.0% [  117

In [65]:
for train_dataset, val_dataset in k_fold_subj.split(dataset):
    # Initialize the model

    model = EEGNet(chunk_size=250,
               num_electrodes=26,
               dropout=0.5,
               kernel_1=64,
               kernel_2=64,
               F1=8,
               F2=16,
               D=2,
               num_classes=2)

    # Initialize the trainer and use the 0-th GPU for training, or set device_ids=[] to use CPU
    trainer = MyClassificationTrainer(model=model,
                                      lr=1e-4,
                                      weight_decay=1e-4,
                                      device_ids=[0])

    # Initialize several batches of training samples and test samples
    train_loader = DataLoader(train_dataset,
                              batch_size=25,
                              shuffle=True,
                              num_workers=6)
    val_loader = DataLoader(val_dataset,
                            batch_size=25,
                            shuffle=False,
                            num_workers=6)

    # Do 50 rounds of training
    trainer.fit(train_loader, val_loader, num_epochs=100)
    trainer.test(val_loader)

Epoch 1
-------------------------------
Epoch 1
-------------------------------
loss: 0.632676, accuracy: 60.0% [    0/  191]
loss: 0.632676, accuracy: 60.0% [    0/  191]
loss: 0.638930, accuracy: 60.0% [   39/  191]
loss: 0.638930, accuracy: 60.0% [   39/  191]
loss: 0.752188, accuracy: 48.0% [   78/  191]
loss: 0.752188, accuracy: 48.0% [   78/  191]
loss: 0.738073, accuracy: 44.0% [  117/  191]
loss: 0.738073, accuracy: 44.0% [  117/  191]
loss: 0.809251, accuracy: 40.0% [  156/  191]
loss: 0.809251, accuracy: 40.0% [  156/  191]

loss: 0.695049, accuracy: 54.2%

loss: 0.695049, accuracy: 54.2%
Epoch 2
-------------------------------
Epoch 2
-------------------------------
loss: 0.669047, accuracy: 52.0% [    0/  191]
loss: 0.669047, accuracy: 52.0% [    0/  191]
loss: 0.695891, accuracy: 52.0% [   39/  191]
loss: 0.695891, accuracy: 52.0% [   39/  191]
loss: 0.704403, accuracy: 48.0% [   78/  191]
loss: 0.704403, accuracy: 48.0% [   78/  191]
loss: 0.798113, accuracy: 40.0% [  117